# [에어크루] 항공사 고객 만족도 예측 경진대회 
# Stage 6. 모델 학습 강화를 통한 정확도 향상

항공사 고객 만족도 예측 경진대회에 참여하신 데이커 여러분, 환영합니다. 🙌  
이번 스테이지에서는 모델링 고도화에 대해 알아보겠습니다

---

## 6. 모델 학습 강화

모델 앙상블을 통한 모델링 고도화를 진행해보겠습니다.

🔎 **모델 앙상블이란 ?**

데이터를 통해 값을 예측 한다고 할 때, 보통은 하나의 모델을 사용하지만, 모델 앙상블을 통해 **여러 개의 모델을 학습** 시켜 

그 모델의 예측 결과를 통합하여 정확도를 향상시킬 수 있습니다.

![000model_image](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fen4sm9%2FbtqGqQXDrv8%2FKyYd0L8cOAROVV3ckrapZ0%2Fimg.png)

출처 : https://dokhakdubini.tistory.com/237

앙상블 학습에도 종류가 다양한데요. 이번에는 ‘voting’을 이용해보려 합니다.

🔎 **voting** 이란 다른 알고리즘이 낸 결과물에 대하여 투표를 통해 결정하는 방식입니다. 

🔎 **voting**을 사용하면 어떤 장점이 있을까요? 

- 과적합(overfitting) 감소 효과
- 개별 모델 성능이 잘 안 나올 때 앙상블 학습을 이용하면 성능 향상

모델을 돌리기 전에 shape 메서드를 통해 각각 형태를 확인해볼게요!

In [11]:
train_x.shape, train_y.shape, test_x.shape

((3000, 22), (3000,), (2000, 22))

Stage4. Baseline modeling에서 RandomForest 모델은 한 번 사용해봤으니, 새로운 ExtraTree 모델을 추가해서 voting을 해 볼게요! 먼저, import부터✈️

🔎 ExtraTree 모델이 샘플 형성 시, 무작위성이 RandomForest 보다 강하므로 함께 voting 해 줍니다!

해당 모델에 적용되는 파라미터는 임의 설정값입니다. max_features만 앞서 확인한 shape과 동일하게 맞춰줬어요.

In [12]:
#ExtraTreesClassifier, RandomForestClassifier 선택

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, VotingClassifier

et_cls = ExtraTreesClassifier(n_estimators=500, min_samples_leaf=5, min_samples_split=7, max_features=22)
rf_cls = RandomForestClassifier(n_estimators=500, min_samples_leaf=5, min_samples_split=7, max_features=22)

# 모델 voting
voting = VotingClassifier(
    estimators=[
        ('et', et_cls),
        ('rf', rf_cls),
    ]
)
voting.fit(train_x, train_y)

VotingClassifier(estimators=[('et',
                              ExtraTreesClassifier(max_features=22,
                                                   min_samples_leaf=5,
                                                   min_samples_split=7,
                                                   n_estimators=500)),
                             ('rf',
                              RandomForestClassifier(max_features=22,
                                                     min_samples_leaf=5,
                                                     min_samples_split=7,
                                                     n_estimators=500))])

예측 결과를 제출 파일에 대입하면 끝입니다.

In [14]:
pred = voting.predict(test_x)
sub['target'] = pred

sub.head()

,id,target
0,1,1
1,2,0
2,3,1
3,4,0
4,5,1


In [15]:
sub.to_csv('answer.csv', index = False)

---
### 전체 흐름 다시 훑어보기

##### 1. 패키지 불러오기

In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import csv
import numpy as np
import seaborn as sns

##### 2. 데이터 불러오기

In [9]:
#변수명 = pd.read_csv(파일명)
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
sub = pd.read_csv('./sample_submission.csv')

##### 3. 열 선택하기

In [3]:
train_x = train.drop(['id','target'],axis=1) 
train_y = train.target 
test_x = test.drop(['id'],axis = 1)

##### 4. 전처리_Label Encoding

In [4]:
from sklearn.preprocessing import LabelEncoder

# qualitative to quantitative
qual_col = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i]) #train_x와 test_x의 Label이 다르게 Encoding될 우려가 있어 train_x로 fit을 고정
    train_x[i] = le.transform(train_x[i])

    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


##### 4. 전처리_데이터 타입 변화

In [5]:
train_x = train_x.astype({'Arrival Delay in Minutes':'int'})

##### 4. 전처리_정규화

In [6]:
from sklearn.preprocessing import StandardScaler

num_features = ['Flight Distance','Departure Delay in Minutes','Arrival Delay in Minutes']

scaler = StandardScaler()

#Data Leakage가 일어나지 않도록 train_x를 통해 스케일러를 학습 시키고 (fit)
scaler.fit(train_x[num_features])

#만들어진 스케일러로 train_x를 정규화해줍니다.(transform)
train_x[num_features] = scaler.transform(train_x[num_features])

##### 5. 모델링

In [7]:
#ExtraTreesClassifier, RandomForestClassifier 선택

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, VotingClassifier

et_cls = ExtraTreesClassifier(n_estimators=500, min_samples_leaf=5, min_samples_split=7, max_features=22)
rf_cls = RandomForestClassifier(n_estimators=500, min_samples_leaf=5, min_samples_split=7, max_features=22)

# 모델 voting
voting = VotingClassifier(
    estimators=[
        ('et', et_cls),
        ('rf', rf_cls),
    ]
)
voting.fit(train_x, train_y)

VotingClassifier(estimators=[('et',
                              ExtraTreesClassifier(max_features=22,
                                                   min_samples_leaf=5,
                                                   min_samples_split=7,
                                                   n_estimators=500)),
                             ('rf',
                              RandomForestClassifier(max_features=22,
                                                     min_samples_leaf=5,
                                                     min_samples_split=7,
                                                     n_estimators=500))])

##### 6. 제출 파일 생성

In [10]:
pred = voting.predict(test_x)
sub['target'] = pred

sub.to_csv('answer.csv', index=False)

---

stage 6, 무사히 완료 =33

다음 stage에서 만나고 싶지만..

이제 stage 끝!✈️

PBL stage 완주를 축하드립니다 🙌